# Parse psl files to get a list of names

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt

In [8]:
dd = pd.read_csv('output.psl', sep='\t', header=0)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,1,2,3,4,5,6,7,10,11,12,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
list(dd) # >=200, Q>

['match',
 'mis- ',
 'rep. ',
 "N's",
 'Q gap',
 'Q gap.1',
 'T gap',
 'T gap.1',
 'strand',
 'Q        ',
 'Q   ',
 'Q    ',
 'Q  ',
 'T        ',
 'T   ',
 'T    ',
 'T  ',
 'block',
 'blockSizes ',
 'qStarts',
 ' tStarts']

In [10]:
dd.head(2)

,match,mis-,rep.,N's,Q gap,Q gap.1,T gap,T gap.1,strand,Q,...,Q,Q,T,T,T,T,block,blockSizes,qStarts,tStarts
0,,match,match,,count,bases,count,bases,,name,...,start,end,name,size,start,end,count,NaN,NaN,NaN
1,283,1,0,0,5,22,11,1321,+,3e701bd-7e1f-4d25-a5de-cf2af3815b83_637,...,72,378,NG_029465.2:5001-114630,109630,106810,108415,14,"15,14,24,11,9,25,6,62,3,12,23,32,33,15,","72,91,105,129,140,149,186,192,254,257,271,297,...","106810,106832,106848,106873,108170,108180,1082..."


In [11]:
dd = dd.rename(index=str, columns={'mis- ': 'mismatch', 'rep. ': 'repmatch', 'Q gap':"Q gap count", "Q gap.1":"Q gap bases",
                             "T gap":"T gap count", "T gap.1":"T gap bases", "Q        ":"Q name", 
                             "Q   ":"Q size", "Q    ":"Q start", "Q  ":"Q end", "T        ":"T name",
                             "T   ":"T size", "T    ":"T start", "T  ":"T end", "block":"block count",
                             "blockSizes ":"blockSizes", ' tStarts':'tStarts'})

In [12]:
dd = dd.drop(dd.index[[0]])
dd = dd.reset_index()

In [13]:
dd.head()

,index,match,mismatch,repmatch,N's,Q gap count,Q gap bases,T gap count,T gap bases,strand,...,Q start,Q end,T name,T size,T start,T end,block count,blockSizes,qStarts,tStarts
0,1,283,1,0,0,5,22,11,1321,+,...,72,378,NG_029465.2:5001-114630,109630,106810,108415,14,"15,14,24,11,9,25,6,62,3,12,23,32,33,15,","72,91,105,129,140,149,186,192,254,257,271,297,...","106810,106832,106848,106873,108170,108180,1082..."
1,2,134,7,0,0,2,38,2,43049,+,...,439,618,NG_029465.2:5001-114630,109630,9080,52270,3,"12,106,23,","439,472,595,","9080,9116,52247,"
2,3,98,7,0,0,1,17,1,16,+,...,493,615,NG_029465.2:5001-114630,109630,23452,23573,2,"85,20,","493,595,","23452,23553,"
3,4,162,10,0,0,1,3,6,40059,-,...,447,622,NG_029465.2:5001-114630,109630,59208,99439,7,"25,20,18,3,56,37,13,","556,581,604,622,625,681,718,","59208,71243,71280,71353,71613,75118,99426,"
4,5,128,9,0,0,2,46,2,192,-,...,439,622,NG_029465.2:5001-114630,109630,81130,81459,3,"28,97,12,","556,601,727,","81130,81174,81447,"


In [14]:
names = list(set(dd['Q name']))

In [15]:
with open('Names.txt', 'w') as f:
    for item in list(names):
        f.write("%s\n" % item)

read name list

In [5]:
f = open('Names.txt', 'r')
names = f.readlines()
f.close()

In [16]:
print(len(names))

93543


# Parse fastq file

In [10]:
f = open('allPreSplint.fastq', 'r')
qq = f.readlines()
f.close()

In [15]:
qq[0:4]

['@b3e701bd-7e1f-4d25-a5de-cf2af3815b83_637\n',
 'CTCAGTATTACTTCGTTCAGTTACGTATTGCTCAAATATGGAACTCATCAGCCTCATATATTCGATCTGACACAGGGCAAAGACCAGTACTCTGTCTTCATGAATGAAAGAGGACATCTTATGGTGCGCGAAATGGAAAGGGTATTTGGTTCCCAGTCCACTATACTGACGTCTTAACCAGCGACACTTGGCGGCAGAGACTGCTGGGCCGGTCATGGAGCGTGCCAGTCATCCGCCACCTCTTCGCTCCGCTGGTATTTGCGTGTGTGCTTAAGGGACATGGGGGCAAACTGAAATTAGCGACACAAAGTTAAACAAACAAACAAAAACACAAAACATAATAAAACACCAAGAACATGAGGGATGGAGAGAAGTATCTTTCAAAAAAAAAAAAAAAAAAAAAAAAAAAAACGCTGCCGCCAGACAGAAGACGTACTCTGCGTTGATACCACTGCTTCCGCGGACAGGCGTGTAGATCTCGGTGGTCGCCGTATCACATTAAAGGGATATTTTCGATCCTTGTTTCAGTTTCGTTTAGTGCATTTGATCCTTTACTCCTCTAAGACAACCTGACCCAGCAAAAGGTACACAATACTTTTATATTGGACTCAAGAATGCTGCCAGCAATGGTTTAAAATCACTAGGCGATCTCACTTAGAATTCATTTTCAGCATTTATTGGTTGTATGAGAGTAGATGAAAAGACAACTCTGGCTTGAAGCTATCAAAAAACTAAGTAGTGATGAAATGGGCGAGTCTACAAATGGACTGCTGCTGGTCAATAAAAGTGACCAATACTAGTCAAACATTCATCCAAGCCTGCGCTGCTGCCTCCTGTATGCGGCGACCACCAGCGTCCGCTAGCTGCAAGCCAATGCTCTCCATCTCATGTTCTTGGTGTTTATTGTGTTTTGTGTTTTGTTGTTCGTTTAACTTTGTGTCACCTGTCAG

In [21]:
tt = pd.DataFrame(list(zip(qq[::4], qq[1::4], qq[3::4])), 
               columns =['Name', 'Sequence', 'Qscore']) #(541486, 3)

In [39]:
tt.head()

,Name,Sequence,Qscore
0,3e701bd-7e1f-4d25-a5de-cf2af3815b83_637,CTCAGTATTACTTCGTTCAGTTACGTATTGCTCAAATATGGAACTC...,'--&'+))*)11<7680<)++-'&%/)0../;;9>=+'(**6038:...
1,ffcb8a3-a78b-4d98-b40b-f79eaafb47c5_259,CTGGTAATACTTCCGTTCAGTTTACGTGTGCTAGCTCGTTCCCATG...,"%-&*,%""(-0563$)17.$0,,'#$$'$(*/010001::>4<9......"
2,2825bff-20d3-4cc2-8ba5-506e97e23423_700,ACCAGTAGCCTTCGTTCAGTTACGTATTGCTTAGTAGAATCTTTTA...,"#""##$(#""""##'..57<.11;13,4254.+.--057.*,+,.:>53..."
3,2d6397c-61b1-4780-9588-aa74ba07921b_311,ATCAGTACTTCGTTCAGTTACGTATTGCTGGGGGCCGAGGCAGGTC...,"$.#""#'%-.>;;;2;&&'=/++%%)+)*.:==;33<+*+,2#&'&(..."
4,f1dbfaf-f271-4fb1-a4f5-d773bbc19168_236,ATCATTGTACTTCGTTCGGTTACGTATTGCTGGTACTCTGCGTTGA...,"""""""&+/),'/80<:;<:&8,=.//62:=9;;>92/1/1446728++..."


In [28]:
tt['Name'] = tt['Name'].str[1:-1]
tt['Sequence'] = tt['Sequence'].str[:-1]
tt['Qscore'] = tt['Qscore'].str[:-1]

In [51]:
tt.sort_values('name')
names.sort()

In [45]:
#tt['Name'].str.split('_', 1, expand=True)
tt['name'], tt['add'] = tt['Name'].str.split('_', 1).str

In [52]:
tt.head()

,Name,Sequence,Qscore,name,add
0,3e701bd-7e1f-4d25-a5de-cf2af3815b83_637,CTCAGTATTACTTCGTTCAGTTACGTATTGCTCAAATATGGAACTC...,'--&'+))*)11<7680<)++-'&%/)0../;;9>=+'(**6038:...,3e701bd-7e1f-4d25-a5de-cf2af3815b83,637
1,ffcb8a3-a78b-4d98-b40b-f79eaafb47c5_259,CTGGTAATACTTCCGTTCAGTTTACGTGTGCTAGCTCGTTCCCATG...,"%-&*,%""(-0563$)17.$0,,'#$$'$(*/010001::>4<9......",ffcb8a3-a78b-4d98-b40b-f79eaafb47c5,259
2,2825bff-20d3-4cc2-8ba5-506e97e23423_700,ACCAGTAGCCTTCGTTCAGTTACGTATTGCTTAGTAGAATCTTTTA...,"#""##$(#""""##'..57<.11;13,4254.+.--057.*,+,.:>53...",2825bff-20d3-4cc2-8ba5-506e97e23423,700
3,2d6397c-61b1-4780-9588-aa74ba07921b_311,ATCAGTACTTCGTTCAGTTACGTATTGCTGGGGGCCGAGGCAGGTC...,"$.#""#'%-.>;;;2;&&'=/++%%)+)*.:==;33<+*+,2#&'&(...",2d6397c-61b1-4780-9588-aa74ba07921b,311
4,f1dbfaf-f271-4fb1-a4f5-d773bbc19168_236,ATCATTGTACTTCGTTCGGTTACGTATTGCTGGTACTCTGCGTTGA...,"""""""&+/),'/80<:;<:&8,=.//62:=9;;>92/1/1446728++...",f1dbfaf-f271-4fb1-a4f5-d773bbc19168,236


In [57]:
kk = tt[tt['name'].isin(names)]
print(kk.shape)
kk.head()

(0, 5)


,Name,Sequence,Qscore,name,add


In [55]:
names = [x[:-1] for x in names]

In [58]:
tt.to_csv("allSeq_fastq.csv")

In [59]:
#write to fasta
with open('allPreSplintFasta.fasta', 'w') as f:
    for index, row in tt.iterrows():
        f.write(">%s\n" % row[0])
        f.write("%s\n" % row[1])

# Read in FastQ

In [18]:
bb = pd.read_csv('allSeq_fastq.csv')

In [19]:
print(bb.shape)
bb.head()

(541486, 6)


,Unnamed: 0,Name,Sequence,Qscore,name,add
0,0,3e701bd-7e1f-4d25-a5de-cf2af3815b83_637,CTCAGTATTACTTCGTTCAGTTACGTATTGCTCAAATATGGAACTC...,'--&'+))*)11<7680<)++-'&%/)0../;;9>=+'(**6038:...,3e701bd-7e1f-4d25-a5de-cf2af3815b83,637
1,1,ffcb8a3-a78b-4d98-b40b-f79eaafb47c5_259,CTGGTAATACTTCCGTTCAGTTTACGTGTGCTAGCTCGTTCCCATG...,"%-&*,%""(-0563$)17.$0,,'#$$'$(*/010001::>4<9......",ffcb8a3-a78b-4d98-b40b-f79eaafb47c5,259
2,2,2825bff-20d3-4cc2-8ba5-506e97e23423_700,ACCAGTAGCCTTCGTTCAGTTACGTATTGCTTAGTAGAATCTTTTA...,"#""##$(#""""##'..57<.11;13,4254.+.--057.*,+,.:>53...",2825bff-20d3-4cc2-8ba5-506e97e23423,700
3,3,2d6397c-61b1-4780-9588-aa74ba07921b_311,ATCAGTACTTCGTTCAGTTACGTATTGCTGGGGGCCGAGGCAGGTC...,"$.#""#'%-.>;;;2;&&'=/++%%)+)*.:==;33<+*+,2#&'&(...",2d6397c-61b1-4780-9588-aa74ba07921b,311
4,4,f1dbfaf-f271-4fb1-a4f5-d773bbc19168_236,ATCATTGTACTTCGTTCGGTTACGTATTGCTGGTACTCTGCGTTGA...,"""""""&+/),'/80<:;<:&8,=.//62:=9;;>92/1/1446728++...",f1dbfaf-f271-4fb1-a4f5-d773bbc19168,236


In [25]:
kk = bb[bb['Name'].isin(names)]

In [27]:
print(kk.shape)
kk.head()

(187086, 6)


,Unnamed: 0,Name,Sequence,Qscore,name,add
0,0,3e701bd-7e1f-4d25-a5de-cf2af3815b83_637,CTCAGTATTACTTCGTTCAGTTACGTATTGCTCAAATATGGAACTC...,'--&'+))*)11<7680<)++-'&%/)0../;;9>=+'(**6038:...,3e701bd-7e1f-4d25-a5de-cf2af3815b83,637
4,4,f1dbfaf-f271-4fb1-a4f5-d773bbc19168_236,ATCATTGTACTTCGTTCGGTTACGTATTGCTGGTACTCTGCGTTGA...,"""""""&+/),'/80<:;<:&8,=.//62:=9;;>92/1/1446728++...",f1dbfaf-f271-4fb1-a4f5-d773bbc19168,236
9,9,db68828-7ad5-47c2-b2de-edfee3141c16_1042,CTGTGTACTTCGTTCAGTTACGTATTGCTCACCTCTTCGCTCCGCT...,"$##%$&(,)+0/1:8788<883:5693460/09)-0;77755<227...",db68828-7ad5-47c2-b2de-edfee3141c16,1042
10,10,247352a-7bf9-4384-9154-c4fadd6f540e_249,CTCAATTGTACTTCGTTCAGTTACGTAGTGCACCATAAGATCTCCT...,"#""&'""/#$''*+3+,079(-6+#'#&""$$-/**%$(')'*+'+.4+...",247352a-7bf9-4384-9154-c4fadd6f540e,249
12,12,09e4d6f-1151-4ff0-b17c-55a854ea75df_492,ATCAGTACTTCGTTCAGTTACGTATTGCTCGCGGAAACGTGGGTAT...,"""%$#""""""#&1.1726&104**(,&)+*.::7<8-/*"",*0190*-....",09e4d6f-1151-4ff0-b17c-55a854ea75df,492


In [30]:
i=100
len(kk['Sequence'][i]) == len(kk['Qscore'][i])

True

# Write to fastQ

In [31]:
with open('filtered.fastq', 'w') as f:
    for index, row in kk.iterrows():
        f.write("@%s\n" % row[1])
        f.write("%s\n" % row[2])
        f.write("+\n")
        f.write("%s\n" % row[3])

# Parse fasta file

In [16]:
bb = pd.read_csv('preout.fasta', header=None)

In [17]:
bb.head()

,0
0,>25e9f11c-c86b-41a1-98ee-50885d425a18
1,ATCAGTATGCTTCGTTCGGTTACGTATTGCTCCACCCACTTCTCTC...
2,>b3e701bd-7e1f-4d25-a5de-cf2af3815b83
3,CTCAGTATTACTTCGTTCAGTTACGTATTGCTCAAATATGGAACTC...
4,>cffcb8a3-a78b-4d98-b40b-f79eaafb47c5


In [18]:
tt = pd.DataFrame(list(zip(bb.iloc[::2].values.reshape(-1), bb.iloc[1::2].values.reshape(-1))), 
               columns =['Name', 'Sequence']) #464608

In [19]:
tt.head()

,Name,Sequence
0,>25e9f11c-c86b-41a1-98ee-50885d425a18,ATCAGTATGCTTCGTTCGGTTACGTATTGCTCCACCCACTTCTCTC...
1,>b3e701bd-7e1f-4d25-a5de-cf2af3815b83,CTCAGTATTACTTCGTTCAGTTACGTATTGCTCAAATATGGAACTC...
2,>cffcb8a3-a78b-4d98-b40b-f79eaafb47c5,CTGGTAATACTTCCGTTCAGTTTACGTGTGCTAGCTCGTTCCCATG...
3,>72825bff-20d3-4cc2-8ba5-506e97e23423,ACCAGTAGCCTTCGTTCAGTTACGTATTGCTTAGTAGAATCTTTTA...
4,>1350df46-ebb9-4f01-a8a5-ab279c30327b,CTCGGTGCTAGTTCAGTTACGTATTGCTCCACCAGGGCTTCTTCTG...


In [23]:
tt['Name'].head()

0    25e9f11c-c86b-41a1-98ee-50885d425a18
1    b3e701bd-7e1f-4d25-a5de-cf2af3815b83
2    cffcb8a3-a78b-4d98-b40b-f79eaafb47c5
3    72825bff-20d3-4cc2-8ba5-506e97e23423
4    1350df46-ebb9-4f01-a8a5-ab279c30327b
Name: Name, dtype: object

In [22]:
tt['Name'] = tt['Name'].str[1:]

In [24]:
tt.to_csv("allSeqs.csv")

In [26]:
print(tt.shape)
print(len(names))

(464608, 2)
127384


In [27]:
tt.sort_values('Name')

,Name,Sequence
428964,0000038e-4753-48fe-9c8b-5aba9af03220,CTCATTGTGCTTCGTTCGTTTACGTATTGCTAAGATGAGAAGAGGA...
208737,00001b99-67d4-47fa-8cf6-5164170e8a5b,CTCATTGTACTTCGTTCAGTTACGTATTGCTCATTTGGATTAGCGT...
38797,00002836-8a2a-488a-8fbb-04789ecc04fd,CTCAATTGTACTTCGTTCAGTTACATTGCTGTAATGCAAATTTTTA...
6901,000044fa-33f5-47dd-8a41-d1c96f1fb15b,ACAGTATGCTTCGTTTCGGTACGTATTGCTAGAGGCCCATGTCTGC...
74844,00005bd1-0518-4feb-9313-a02c6a0d98b5,CTGTTAATACTTCGTTTCGGTTACGTATTGCTAAAGGGTGGGATCG...
283713,000073a0-9d76-440c-a94f-a7bdf9973801,CTCGGTAGCCGTTCAGTTGCATTGCTCAGCTGTGGTTTTAAAGCTG...
422115,000082d8-03c9-4827-8904-958ecbe3e94a,TCGGTACTTCGTTCCGGTTACGTATTGCTGAGTACGTCTTCTGTCT...
448739,0000c561-c8c0-491d-b058-bc7230d31712,CTCAGTGGCCTTTCGTTCCAGTTACGTATTGCTCAGGCAAATTTCT...
293336,0000c828-767f-4f5a-b92a-6e3e49d32d61,CTCAGTTGTGTGCTATCGAGGGGATGGCACCACTGCAGTGGCAGTG...
298148,0000ffde-73b9-466d-9cb9-a5e895acf654,ACCATTTGCTTCGTTCAGTTACGTATTGCTGAAGGGTGGCGGATGA...


In [29]:
names = list(names)
names.sort()

In [30]:
names

['0000038e-4753-48fe-9c8b-5aba9af03220',
 '00005bd1-0518-4feb-9313-a02c6a0d98b5',
 '0000ffde-73b9-466d-9cb9-a5e895acf654',
 '00012dd7-dfd4-453b-9fbc-9be0e4868169',
 '0001360b-91e2-4637-b3de-5d24b626b6f9',
 '000191d3-3dc0-4bcf-b6aa-5a1d2b2ba7ad',
 '0001b45e-7c1c-4efd-90fe-6246eebdd34a',
 '0001eba1-1598-427a-92a8-930747becc5b',
 '0002aa27-99f4-43d7-9c8c-068702a8d2f4',
 '000302aa-7b7c-4854-bd7e-d40830e79244',
 '0003630c-4ac5-4c38-8276-0e28d699b3dd',
 '0003f291-43d0-4f04-bb5f-5dd47c7d0016',
 '0004259b-ae31-4e67-8c42-690aa0992e5a',
 '000432f5-9e31-40a4-aead-afaa9698d72d',
 '00078ae4-ba81-43fb-b5e6-8bfc1dd1f4eb',
 '0007e186-d449-4e94-98b5-436b827fb09f',
 '0008bfe3-4ee4-4487-bcec-0b3d2bc87796',
 '0008c99b-f61e-4436-bcd6-19facdbcc385',
 '0008dd0e-9018-4848-a61c-9e7da5bfe140',
 '0008e3bc-a33e-4958-94a8-c83abe09f0a2',
 '0008ebbf-47c2-4f41-bb6d-3c21bd73c2c0',
 '0008ee91-28a9-4a67-b44d-7aa1085155ed',
 '00091b5d-6dec-4b86-aa1f-0aaba9b6caa1',
 '000a6c81-caa1-47e2-8d55-a7db0aacc402',
 '000b47b1-f185-

In [32]:
kk = tt[tt['Name'].isin(names)]

In [34]:
print(kk.shape)
kk.head()

(127384, 2)


,Name,Sequence
1,b3e701bd-7e1f-4d25-a5de-cf2af3815b83,CTCAGTATTACTTCGTTCAGTTACGTATTGCTCAAATATGGAACTC...
6,af1dbfaf-f271-4fb1-a4f5-d773bbc19168,ATCATTGTACTTCGTTCGGTTACGTATTGCTGGTACTCTGCGTTGA...
14,8db68828-7ad5-47c2-b2de-edfee3141c16,CTGTGTACTTCGTTCAGTTACGTATTGCTCACCTCTTCGCTCCGCT...
15,1247352a-7bf9-4384-9154-c4fadd6f540e,CTCAATTGTACTTCGTTCAGTTACGTAGTGCACCATAAGATCTCCT...
17,409e4d6f-1151-4ff0-b17c-55a854ea75df,ATCAGTACTTCGTTCAGTTACGTATTGCTCGCGGAAACGTGGGTAT...


# Write to fasta

In [37]:
with open('filtered.fasta', 'w') as f:
    for index, row in kk.iterrows():
        f.write(">%s\n" % row[0])
        f.write("%s\n" % row[1])

# Clean up Fasta

In [2]:
aa = pd.read_csv('consensus.fasta', header=None)

In [4]:
tt = pd.DataFrame(list(zip(aa.iloc[::2].values.reshape(-1), aa.iloc[1::2].values.reshape(-1))), 
               columns =['Name', 'Sequence']) #131152

In [6]:
print(tt.shape)
tt.head()

(131152, 2)


,Name,Sequence
0,>f1dbfaf-f271-4fb1-a4f5-d773bbc19168_13.05_117...,AATCACTAGGCGATCTCATGCAAATTCATTTCAGCATTTATTGGTT...
1,>db68828-7ad5-47c2-b2de-edfee3141c16_15.91_132...,GCGATCTCCGCTTAGAATTCATTTCAGCATTTATTGGTTGTATGAG...
2,>247352a-7bf9-4384-9154-c4fadd6f540e_15.54_120...,TTTAAACTATTGCTGGCAGCATTCTTGAGTCCAATATAAAAAGTAT...
3,>03cf611-28ca-4bbd-93a4-3095e832cce0_14.93_189...,AATAGTTTAAAATCACTAGGCGATCTCCATGAAATTCATTTCAGCA...
4,>bd1c7fa-aa45-4b6f-bc5b-a2a53b1a1313_14.36_165...,TTTAAACTATTGCTGGCAGCATTCTTGAGTCAATATAAAGTATTGT...


In [8]:
len(set(tt['Name']))

65576

In [28]:
oo = tt
oo = oo.drop_duplicates(subset='Name', keep='first')

In [29]:
print(oo.shape)
oo.head()

(65576, 2)


,Name,Sequence
0,>f1dbfaf-f271-4fb1-a4f5-d773bbc19168_13.05_117...,AATCACTAGGCGATCTCATGCAAATTCATTTCAGCATTTATTGGTT...
1,>db68828-7ad5-47c2-b2de-edfee3141c16_15.91_132...,GCGATCTCCGCTTAGAATTCATTTCAGCATTTATTGGTTGTATGAG...
2,>247352a-7bf9-4384-9154-c4fadd6f540e_15.54_120...,TTTAAACTATTGCTGGCAGCATTCTTGAGTCCAATATAAAAAGTAT...
3,>03cf611-28ca-4bbd-93a4-3095e832cce0_14.93_189...,AATAGTTTAAAATCACTAGGCGATCTCCATGAAATTCATTTCAGCA...
4,>bd1c7fa-aa45-4b6f-bc5b-a2a53b1a1313_14.36_165...,TTTAAACTATTGCTGGCAGCATTCTTGAGTCAATATAAAGTATTGT...


In [30]:
with open('consensus_half.fasta', 'w') as f:
    for index, row in oo.iterrows():
        f.write("%s\n" % row[0])
        f.write("%s\n" % row[1])

# Clean up fastq subreads

In [11]:
f = open('subreads.fastq', 'r')
qq = f.readlines()
f.close()

In [12]:
tt = pd.DataFrame(list(zip(qq[::4], qq[1::4], qq[3::4])), 
               columns =['Name', 'Sequence', 'Qscore']) #(1064694, 3)

In [13]:
print(tt.shape)
tt.head()

(1064694, 3)


,Name,Sequence,Qscore
0,@f1dbfaf-f271-4fb1-a4f5-d773bbc19168_1 \n,AATCACTAGGCGATCTCATGCAAATTCATTTCAGCATTTATTGGTT...,"=:1(%'(')+//69914%&""""&)/+5177>97567/3>0-.904/3..."
1,@f1dbfaf-f271-4fb1-a4f5-d773bbc19168_0\n,ATCATTGTACTTCGTTCGGTTACGTATTGCTGGTACTCTGCGTTGA...,"""""""&+/),'/80<:;<:&8,=.//62:=9;;>92/1/1446728++..."
2,@f1dbfaf-f271-4fb1-a4f5-d773bbc19168_2\n,GTCTGGCTGGCGACTGCTCGCAGCTGCCTCCGGGATAACGAGCTCA...,"2((%##$"",$#%""#$""$))/+('&#""'##)%(*'##$$'$#""##%""..."
3,@db68828-7ad5-47c2-b2de-edfee3141c16_1 \n,GCGATCTCCGCTTAGAATTCATTTCAGCATTTATTGGTTGTATGAG...,"4:6798895&(&/'%,*5=78;=,*0$732=5,.5..3/-.9:,-*..."
4,@db68828-7ad5-47c2-b2de-edfee3141c16_0\n,CTGTGTACTTCGTTCAGTTACGTATTGCTCACCTCTTCGCTCCGCT...,"$##%$&(,)+0/1:8788<883:5693460/09)-0;77755<227..."


In [14]:
oo = tt.drop_duplicates(subset='Name', keep='first')
print(oo.shape)

(532347, 3)


In [15]:
oo.head()

,Name,Sequence,Qscore
0,@f1dbfaf-f271-4fb1-a4f5-d773bbc19168_1 \n,AATCACTAGGCGATCTCATGCAAATTCATTTCAGCATTTATTGGTT...,"=:1(%'(')+//69914%&""""&)/+5177>97567/3>0-.904/3..."
1,@f1dbfaf-f271-4fb1-a4f5-d773bbc19168_0\n,ATCATTGTACTTCGTTCGGTTACGTATTGCTGGTACTCTGCGTTGA...,"""""""&+/),'/80<:;<:&8,=.//62:=9;;>92/1/1446728++..."
2,@f1dbfaf-f271-4fb1-a4f5-d773bbc19168_2\n,GTCTGGCTGGCGACTGCTCGCAGCTGCCTCCGGGATAACGAGCTCA...,"2((%##$"",$#%""#$""$))/+('&#""'##)%(*'##$$'$#""##%""..."
3,@db68828-7ad5-47c2-b2de-edfee3141c16_1 \n,GCGATCTCCGCTTAGAATTCATTTCAGCATTTATTGGTTGTATGAG...,"4:6798895&(&/'%,*5=78;=,*0$732=5,.5..3/-.9:,-*..."
4,@db68828-7ad5-47c2-b2de-edfee3141c16_0\n,CTGTGTACTTCGTTCAGTTACGTATTGCTCACCTCTTCGCTCCGCT...,"$##%$&(,)+0/1:8788<883:5693460/09)-0;77755<227..."


In [16]:
with open('subreads_half.fastq', 'w') as f:
    for index, row in oo.iterrows():
        f.write("%s" % row[0])
        f.write("%s" % row[1])
        f.write("+\n")
        f.write("%s" % row[2])

# Get rid of empty strings in R2C2

In [8]:
f = open('R2C2_full_length_consensus_reads_R2.fasta', 'r')
qq = f.readlines()
f.close()

In [9]:
tt = pd.DataFrame(list(zip(qq[::4], qq[1::4])), 
               columns =['Name', 'Sequence']) #(1064694, 3)

In [10]:
print(tt.shape)
tt.head()

(24886, 2)


,Name,Sequence
0,>db68828-7ad5-47c2-b2de-edfee3141c16_15.91_132...,GCTCTTCCGATCTGACGCAGGGCAAAAGACCCAGCATTTTCCTGTC...
1,>03cf611-28ca-4bbd-93a4-3095e832cce0_14.93_189...,ATGCTCTTCCGATCTGACGCAGGGCAAAGACCAGCATTTTCCTGTC...
2,>954efb3-70f9-46ba-88fb-e58b75b21cf0_13.24_195...,ATGCTCTTCCGATCTGACGCAGGGCAAAGACCAGCCTGACCAACAT...
3,>b0f9dc4-aaf3-4cc7-a3cf-e15b794ba0bf_15.72_139...,TGCTTCTTCCGATCTGACACAGGGCAAAGACCAGCATTTTCCTGTC...
4,>dfff674-befc-49ef-9f81-9063984c1571_14.23_174...,ATGCTCTTCCGATCTGACACAGGGCAAAGACCCCAATGCCCAAAGA...


In [16]:
tt[tt['Name'] == '>01551a5-692a-45a3-a215-fe6684317e18_13.63_2630_2_648_275\n']

,Name,Sequence
526,>01551a5-692a-45a3-a215-fe6684317e18_13.63_263...,\n


In [19]:
tt = tt[tt['Sequence'] != '\n']

In [21]:
with open('R2C2_clean.fasta', 'w') as f:
    for index, row in tt.iterrows():
        f.write("%s" % row[0])
        f.write("%s" % row[1])